In [237]:
!pip install openai

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [238]:
from dotenv import load_dotenv
import os

In [239]:
load_dotenv()  # Load environment variables from the .env file

True

In [240]:
!pip install python-dotenv

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [241]:
api_key = os.getenv('OPENAI_API_KEY')

In [242]:
!pip install OpenAI

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: pip install --upgrade pip


# Expression parsing

In [243]:
def extract_structure(latex_expression):
    
    client = OpenAI()
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {
                "role": "system",
                "content": "You are a helpful assistant that parses LaTeX expressions and categorizes their components "
                    "into the following structure: Variables/Constants, Functions, Operators, Brackets, and Misc. "
                    "For each category, provide a list of items found in the LaTeX expression. "
                    "Ensure that any mathematical functions, such as 'max', 'min', 'sin', 'cos', and custom functions like 'U_u', are correctly identified under the 'Functions' category. "
                    "Variables are any symbols or letters that represent quantities, such as 't', 'x', or 'omega_t^o'. "
                    "Constants include numeric values like '-0.20', '0.22', and '0.0361'. "
                    "Operators include symbols like '>', '>=', '+', '-', '*', and '/' that indicate operations between variables or constants. "
                    "Brackets include any grouping symbols such as '(', ')', '[', ']', '{', and '}'. "
                    "Misc should be used for any elements that do not fit into the other categories. "
                    "The output should be a JSON object with the following keys: 'Variables/Constants', 'Functions', 'Operators', 'Brackets', 'Misc'. "
                    "Numbers also fall under constants and variables, and convert the LaTeX into mathematical expressions without LaTeX-specific formatting."
                    "output should be a json dictionary with categories as keys and lists of items as values"
            },
            {
                "role": "user",
                "content": f"Please parse the following LaTeX expression and provide the categorized output structure: {latex_expression}"
            }
        ],
        max_tokens=500,
        temperature=0.5,
    )

    return response.choices[0].message.content

# Provided LaTeX expression
latex_expression = r"t \in [0.000, 0.0361) \rightarrow & \
U_u(\omega_t^o) \geq \max\left(Q_{U_{\Omega^o_t}} (-0.20 \cdot t + 0.22), \bar{u_t}\right)\\"

# Get the output structure from the API
output_structure = extract_structure(latex_expression)
print(output_structure)

{
  "Variables/Constants": ["t", "0.000", "0.0361", "\omega_t^o", "-0.20", "0.22", "\bar{u_t}"],
  "Functions": ["U_u", "max", "Q_{U_{\Omega^o_t}}"],
  "Operators": ["\in", "(", ")", "\geq", "*", "+"],
  "Brackets": ["[", "]", "{", "}"],
  "Misc": ["\\rightarrow", "&", "\\"]
}


In [244]:
output_structure

'{\n  "Variables/Constants": ["t", "0.000", "0.0361", "\\omega_t^o", "-0.20", "0.22", "\\bar{u_t}"],\n  "Functions": ["U_u", "max", "Q_{U_{\\Omega^o_t}}"],\n  "Operators": ["\\in", "(", ")", "\\geq", "*", "+"],\n  "Brackets": ["[", "]", "{", "}"],\n  "Misc": ["\\\\rightarrow", "&", "\\\\"]\n}'

In [245]:
def convert_output_to_latex(output_structure):
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {
                "role": "system",
                "content": "you are given a JSON format which contains "
            },
            {
                "role": "user",
                "content": f"Please parse the following LaTeX expression and provide the categorized output structure: {latex_expression}"
            }
        ],
        max_tokens=500,
        temperature=0.5,
    )

    return response.choices[0].message.content
    



# Extracting the semantic meaning

In [247]:
# parsed_output = extract_structure(latex_expression)
# prompt = (
#         "The following is a JSON object with categorized components of a LaTeX expression. "
#         "Please provide detailed semantic meanings for each item in the JSON. The categories are: "
#         "Variables/Constants, Functions, Operators, Brackets, and Misc. "
#         "For each item, provide its name and a detailed description of its meaning.\n\n"
#         f"{parsed_output}\n\n"
#         "Provide the output in the following JSON format:"
#         "{"
#         '  "Variables/Constants": ['
#         '    {"name": "item_name", "meaning": "item_meaning"}'
#         '  ],'
#         '  "Functions": '
#         '    {"name": "item_name", "meaning": "item_meaning"}'
#         '  ],'
#         '  "Operators": ['
#         '    {"name": "item_name", "meaning": "item_meaning"}'
#         '  ],'
#         '  "Brackets": ['
#         '    {"name": "item_name", "meaning": "item_meaning"}'
#         '  ],'
#         '  "Misc": ['
#         '    {"name": "item_name", "meaning": "item_meaning"}'
#         '  ]'
#         "}"
#     )

def extract_semantics(parsed_output):
    
    prompt_semantics = (
        "The following is a JSON object with categorized components of a LaTeX expression. "
        "Please provide detailed semantic meanings for each item in the JSON. The categories are: "
        "Variables/Constants, Functions, Operators, Brackets, and Misc. "
        "For each item, provide its name and a detailed description of its meaning.\n\n"
        f"{parsed_output}\n\n"
        "Provide the output in the following JSON format:"
        "{"
        '  "Variables/Constants": ['
        '    {"name": "item_name", "meaning": "item_meaning"}'
        '  ],'
        '  "Functions": '
        '    {"name": "item_name", "meaning": "item_meaning"}'
        '  ],'
        '  "Operators": ['
        '    {"name": "item_name", "meaning": "item_meaning"}'
        '  ],'
        '  "Brackets": ['
        '    {"name": "item_name", "meaning": "item_meaning"}'
        '  ],'
        '  "Misc": ['
        '    {"name": "item_name", "meaning": "item_meaning"}'
        '  ]'
        "}"
    )
    
    
    response = client.chat.completions.create(
        model="gpt-4-turbo",
        messages=[
            {
                "role": "system",
                "content": "You are a helpful assistant."
            },
            {
                "role": "user",
                "content": prompt
            }
        ],
        max_tokens=1500,
        temperature=0.5,
    )

    return response.choices[0].message.content

In [248]:
print(extract_semantics(prompt))

Here is the detailed semantic meaning for each item in the JSON based on their categorization in LaTeX expressions:

```json
{
  "Variables/Constants": [
    {"name": "t", "meaning": "Typically represents a variable, often used to denote time."},
    {"name": "0.0361", "meaning": "A constant value, possibly a parameter or a specific measurement."},
    {"name": "1.000", "meaning": "A constant, representing a whole unit or 100% of something."},
    {"name": "u", "meaning": "A variable, possibly representing an unknown quantity or a specific variable in a formula."},
    {"name": "0.10", "meaning": "A constant value, representing a decimal or percentage."},
    {"name": "0.64", "meaning": "Another constant value, also likely representing a decimal or percentage."}
  ],
  "Functions": [
    {"name": "U_u", "meaning": "A function denoted by U with a subscript u, possibly a specific utility function or other mathematical function depending on context."},
    {"name": "max", "meaning": "A ma

In [249]:
output_structure

'{\n  "Variables/Constants": ["t", "0.000", "0.0361", "\\omega_t^o", "-0.20", "0.22", "\\bar{u_t}"],\n  "Functions": ["U_u", "max", "Q_{U_{\\Omega^o_t}}"],\n  "Operators": ["\\in", "(", ")", "\\geq", "*", "+"],\n  "Brackets": ["[", "]", "{", "}"],\n  "Misc": ["\\\\rightarrow", "&", "\\\\"]\n}'

In [257]:
import json

# Example JSON string with unescaped backslashes
output_structure = '''
{
  "Variables/Constants": ["t", "0.0361", "1.000", "\\\\omega_t^o", "u", "Q_{U_{\\\\Omega^o_t}}", "-0.10", "0.64"],
  "Functions": ["U_u", "\\\\max"],
  "Operators": ["\\\\in", "\\\\rightarrow", ">=", "\\\\cdot", "+"],
  "Brackets": ["[", "]", "(", ")", "{"],
  "Misc": ["&"]
}
'''


output_structure = output_structure.replace('\\', '\\\\')

parsed_latex_dict = json.loads(output_structure)

semantic_meanings = {
    "t": "Typically represents a variable for time in equations.",
    "u": "A variable, possibly representing a specific quantity or parameter in a function.",
    "U_u": "Represents a function named 'U' with 'u' as a parameter. This could be a utility function, or another specific mathematical function depending on the context.",
    "max": "A mathematical function that returns the maximum value among its arguments.",
    "Q_{U_{\\Omega^o_t}}": "A function 'Q' applied to another function or variable 'U' which itself depends on a set or parameter 'Ω^o_t'. This could represent a complex mathematical or statistical operation, possibly querying or calculating a specific value from a defined set or distribution.",
    "\\in": "A symbol used in set theory to denote membership; 'x \\in A' means x is an element of set A.",
    ">=": "A relational operator meaning 'greater than or equal to'. It is used to denote inequality between two expressions.",
    "\\cdot": "Represents multiplication in mathematical expressions.",
    "+": "The basic arithmetic operator for addition.",
    "[": "Used to denote the start of an interval or a list in mathematical expressions.",
    "]": "Used to denote the end of an interval or a list in mathematical expressions.",
    "(": "Used to start a grouping of expressions, affecting the order of operations in mathematical expressions.",
    ")": "Used to end a grouping of expressions, affecting the order of operations in mathematical expressions.",
    "{": "Used to start a set or a block of expressions, often used in defining functions or sets.",
    "\\rightarrow": "Denotes a function mapping or an implication in logical expressions. In functions, it can describe the mapping from one set to another.",
    "&": "Represents a logical conjunction (AND) in various contexts."
}

def is_number(s):
    """Check if the string represents a number."""
    try:
        float(s)
        return True
    except ValueError:
        return False

def extract_semantic_meanings(parsed_latex):
    # Initialize the result dictionary
    semantic_extraction = {}

    # Iterate over each category and map the names to their meanings
    for category, items in parsed_latex.items():
        semantic_extraction[category] = []
        for item in items:
            if is_number(item):
                meaning = "A numerical constant, likely representing a specific value or parameter in the equation."
            else:
                meaning = semantic_meanings.get(item, f"No meaning defined for {item}")
            semantic_extraction[category].append({"name": item, "meaning": meaning})

    return semantic_extraction

# Now process the parsed_latex_dict
extracted_meanings = extract_semantic_meanings(parsed_latex_dict)
print(extracted_meanings)


{'Variables/Constants': [{'name': 't', 'meaning': 'Typically represents a variable for time in equations.'}, {'name': '0.0361', 'meaning': 'A numerical constant, likely representing a specific value or parameter in the equation.'}, {'name': '1.000', 'meaning': 'A numerical constant, likely representing a specific value or parameter in the equation.'}, {'name': '\\\\omega_t^o', 'meaning': 'No meaning defined for \\\\omega_t^o'}, {'name': 'u', 'meaning': 'A variable, possibly representing a specific quantity or parameter in a function.'}, {'name': 'Q_{U_{\\\\Omega^o_t}}', 'meaning': 'No meaning defined for Q_{U_{\\\\Omega^o_t}}'}, {'name': '-0.10', 'meaning': 'A numerical constant, likely representing a specific value or parameter in the equation.'}, {'name': '0.64', 'meaning': 'A numerical constant, likely representing a specific value or parameter in the equation.'}], 'Functions': [{'name': 'U_u', 'meaning': "Represents a function named 'U' with 'u' as a parameter. This could be a util

In [254]:
prompt_for_rule_based= f"""
You are an AI model trained to translate mathematical and logical expressions into natural language explanations. Given the semantic output of a mathematical expression, your task is to generate a coherent English explanation. 

The semantic output will include various categories such as Functions, Operators, Variables/Constants, and more. Use the provided information to create a detailed, understandable explanation in plain English.

Here is the semantic output:
{semantic_explanations}

Generate a natural language explanation for this semantic output, making sure to clarify each element's role in the expression.

For example, translate the semantic representation: 
t in [0.000, 0.0361) -> U_u(omega_t^o) >= max(Q_U_Omega_t (-0.20 * t + 0.22), u_t_bar)

into the natural language explanation:  
"At time t, within the range [0.000, 0.0361), the function U_u applied to omega_t^o must yield a value that is greater than or equal to the maximum value between Q_U_Omega_t of (-0.20 * t + 0.22) and the threshold utility u_t_bar."

Please follow a similar structure and level of detail for the provided semantic output.
"""

# Creating Rule based explanations

In [258]:
import re

def generate_rule_based_explanation(semantic_extractions):
    # Start with an empty explanation
    explanation = "Ensures that "

    # Create helper functions to handle specific parts of the explanation
    def handle_functions(semantic_extractions):
        func_explanation = ""
        for func in semantic_extractions['Functions']:
            if func['name'] == "U_u":
                func_explanation += "the utility function U applied to "
            elif func['name'] == "max":
                func_explanation += "the maximum of "
        return func_explanation

    def handle_variables_constants(semantic_extractions):
        var_const_explanation = ""
        for var in semantic_extractions['Variables/Constants']:
            if var['meaning'] == "time":
                var_const_explanation += "time t "
            elif var['meaning'] == "velocity":
                var_const_explanation += "velocity variable"
            elif var['meaning'] == "variable":
                var_const_explanation += f"the variable {var['name']} "
            elif var['meaning'] == "domain":
                var_const_explanation += f"the domain {var['name']} "
            elif var['meaning'] == "utility":
                var_const_explanation += f"a utility threshold {var['name']} "
            elif var['meaning'] == "numeric":
                var_const_explanation += f"the constant value {var['name']} "
        return var_const_explanation

    def handle_operators(semantic_extractions):
        op_explanation = ""
        for op in semantic_extractions['Operators']:
            if op['name'] == ">=":
                op_explanation += "is greater than or equal to "
            elif op['name'] == "->":
                op_explanation += "implies that "
            elif op['name'] == "in":
                op_explanation += "is within the range "
            elif op['name'] == "*":
                op_explanation += "multiplied by "
            elif op['name'] == "+":
                op_explanation += "plus "
        return op_explanation

    # Building the explanation by combining parts
    explanation += handle_functions(semantic_extractions)
    explanation += handle_variables_constants(semantic_extractions)
    explanation += handle_operators(semantic_extractions)

    # Add additional context or wrap-up the sentence if necessary
    explanation += " meets either a calculated statistical value or a specified minimum utility requirement."

    # Return the final explanation
    return explanation

# Example usage with the provided semantic extraction
semantic_extractions = {
    'Brackets': [{'meaning': 'bracket', 'name': '['},
                 {'meaning': 'bracket', 'name': ']'},
                 {'meaning': 'bracket', 'name': '('},
                 {'meaning': 'bracket', 'name': ')'}],
    'Functions': [{'meaning': 'function', 'name': 'U_u'},
                  {'meaning': 'function', 'name': 'max'}],
    'Misc': [{'meaning': 'coordinating conjunction, connecting elements of equal rank', 'name': '&'}],
    'Operators': [{'meaning': 'operator', 'name': 'in'},
                  {'meaning': 'operator', 'name': '->'},
                  {'meaning': 'operator', 'name': '>='},
                  {'meaning': 'operator', 'name': '*'},
                  {'meaning': 'operator', 'name': '+'}],
    'Variables/Constants': [{'meaning': 'time', 'name': 't '},
                            {'meaning': 'numeric', 'name': '0.000'},
                            {'meaning': 'numeric', 'name': '0.0361'},
                            {'meaning': 'variable', 'name': 'omega_t^o'},
                            {'meaning': 'domain', 'name': 'Q_U_Omega_t'},
                            {'meaning': 'utility', 'name': 'u_t_bar'},
                            {'meaning': 'numeric', 'name': '-0.20'},
                            {'meaning': 'numeric', 'name': '0.22'}]
}

# Generate the rule-based explanation
explanation = generate_rule_based_explanation(semantic_extractions)
print(explanation)


Ensures that the utility function U applied to the maximum of time t the constant value 0.000 the constant value 0.0361 the variable omega_t^o the domain Q_U_Omega_t a utility threshold u_t_bar the constant value -0.20 the constant value 0.22 is within the range implies that is greater than or equal to multiplied by plus  meets either a calculated statistical value or a specified minimum utility requirement.


In [ ]:
#evaluating rule based explanation

In [ ]:
#add a function to compare spacy and llm for semantic extraction

In [259]:
import json

  # This function is used to extract the semantic meaning
def generate_rule_based_explanation_usingGPT(semantic_extraction):
    
    # Format the semantic extraction into a JSON string
    prompt = f"""
Given the following semantic extraction of a parsed mathematical expression:

{json.dumps(semantic_extraction, indent=2)}

Based on the above semantic extraction, generate a rule-based explanation for the mathematical expression. The explanation should be clear, concise, and similar in structure to the following example:

"Ensures that \( U_u(\omega_t^o) \) must meet either a fixed utility threshold \( u \) or another calculated statistical measure \( Q \) in the subsequent time interval [0.0361, 1.000]."

The explanation should be simple, straightforward, and avoid unnecessary elaboration. Focus on matching the structure and brevity of the provided example.
"""

    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {
                "role": "system",
                "content": "You are an expert in analysis. You generate easy to understand rule-based explanations for mathematical expressions based on their semantic meanings "
            },
            {
                "role": "user",
                "content": prompt
            }
        ],
        max_tokens=500,
        temperature=0.5,
    )
    
    # Return the generated explanation
    return response.choices[0].message.content

    
    
    

In [260]:
semantic_extraction = extract_semantics(prompt_semantics)
print(generate_rule_based_explanation_usingGPT(semantic_extraction))

"Calculates the maximum value of the utility function \( U_u \) applied to a specific state \( \omega_t^o \) within the range [0.10, 0.64]."


In [ ]:
semantic_extractions = extract_semantics(output_structure) 

In [261]:
import json

# Example string representing the parsed LaTeX dictionary
parsed_latex_str = '{"Variables/Constants": ["t", "0.0361", "1.000", "u", "0.10", "0.64"], "Functions": ["U_u", "max", "Q_{U_{\\\\Omega^o_t}}"], "Operators": ["\\\\in", ">=", "\\\\cdot", "+"], "Brackets": ["[", "]", "(", ")", "{"], "Misc": ["\\\\rightarrow", "&"]}'

# Convert the string to a dictionary
parsed_latex_dict = json.loads(output_structure)

print(parsed_latex_dict)

{'Variables/Constants': ['t', '0.0361', '1.000', '\\\\omega_t^o', 'u', 'Q_{U_{\\\\Omega^o_t}}', '-0.10', '0.64'], 'Functions': ['U_u', '\\\\max'], 'Operators': ['\\\\in', '\\\\rightarrow', '>=', '\\\\cdot', '+'], 'Brackets': ['[', ']', '(', ')', '{'], 'Misc': ['&']}


In [262]:
print(extract_semantics(prompt_semantics))

{
  "Variables/Constants": [
    {"name": "t", "meaning": "Typically represents a variable for time in equations."},
    {"name": "0.0361", "meaning": "A constant value, possibly a parameter or a specific measurement in the context of the equation."},
    {"name": "1.000", "meaning": "A constant, representing a whole unit or 100% of a quantity."},
    {"name": "u", "meaning": "A variable that could represent a specific quantity or parameter, such as speed, utility, or another measurable entity depending on the context."},
    {"name": "0.10", "meaning": "A constant value, representing a decimal or a percentage, possibly used as a rate or proportion in calculations."},
    {"name": "0.64", "meaning": "Another constant value, likely representing a fixed number in an equation or formula."}
  ],
  "Functions": [
    {"name": "U_u", "meaning": "A function possibly representing a utility function dependent on the variable 'u'."},
    {"name": "max", "meaning": "A mathematical function that r

# Evaluation

In [263]:
import nltk
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

## Evaluation of mathematical parsing

In [264]:
ground_truth = {
  "Variables/Constants": ["t", "0.0361", "1.000", "\\omega_t^o", "u", "Q_{U_{\\Omega^o_t}}", "-0.10", "0.64"],
  "Functions": ["U_u", "max"],
  "Operators": ["\\in", "\\rightarrow", "\\geq", "\\cdot", "+"],
  "Brackets": ["[", "]", "(", ")", "\\left(", "\\right)"],
  "Misc": []
}

parsed_expression = extract_structure(latex_expression)


In [265]:
from collections import Counter

def calculate_metrics(ground_truth, parsed_output):
    metrics = {}

    for category in ground_truth:
        gt_set = set(ground_truth[category])
        po_set = set(parsed_output.get(category, []))

        # True Positives (correctly identified)
        true_positives = gt_set.intersection(po_set)
        
        # False Positives (incorrectly identified as present)
        false_positives = po_set - gt_set
        
        # False Negatives (missed by the parser)
        false_negatives = gt_set - po_set

        precision = len(true_positives) / (len(true_positives) + len(false_positives)) if len(true_positives) + len(false_positives) > 0 else 0
        recall = len(true_positives) / (len(true_positives) + len(false_negatives)) if len(true_positives) + len(false_negatives) > 0 else 0
        f1_score = (2 * precision * recall) / (precision + recall) if precision + recall > 0 else 0

        metrics[category] = {
            "Precision": precision,
            "Recall": recall,
            "F1-Score": f1_score,
            "True Positives": list(true_positives),
            "False Positives": list(false_positives),
            "False Negatives": list(false_negatives)
        }

    return metrics

# Calculate metrics
parsed_expression = json.loads(parsed_expression)
metrics = calculate_metrics(ground_truth, parsed_expression)

# Display results
for category, values in metrics.items():
    print(f"Category: {category}")
    print(f"  Precision: {values['Precision']:.2f}")
    print(f"  Recall: {values['Recall']:.2f}")
    print(f"  F1-Score: {values['F1-Score']:.2f}")
    print(f"  True Positives: {values['True Positives']}")
    print(f"  False Positives: {values['False Positives']}")
    print(f"  False Negatives: {values['False Negatives']}")
    print()

Category: Variables/Constants
  Precision: 0.43
  Recall: 0.38
  F1-Score: 0.40
  True Positives: ['\\omega_t^o', '0.0361', 't']
  False Positives: ['-0.20', '0.22', '0.000', '\\bar{u_t}']
  False Negatives: ['1.000', '-0.10', 'u', '0.64', 'Q_{U_{\\Omega^o_t}}']

Category: Functions
  Precision: 0.33
  Recall: 0.50
  F1-Score: 0.40
  True Positives: ['U_u']
  False Positives: ['Q_{U_{\\Omega^o_t}}', '\\max']
  False Negatives: ['max']

Category: Operators
  Precision: 0.83
  Recall: 1.00
  F1-Score: 0.91
  True Positives: ['\\in', '\\rightarrow', '\\cdot', '\\geq', '+']
  False Positives: [',']
  False Negatives: []

Category: Brackets
  Precision: 0.67
  Recall: 0.67
  F1-Score: 0.67
  True Positives: ['(', ')', ']', '[']
  False Positives: ['}', '{']
  False Negatives: ['\\left(', '\\right)']

Category: Misc
  Precision: 0.00
  Recall: 0.00
  F1-Score: 0.00
  True Positives: []
  False Positives: ['\\left', '&', '\\right']
  False Negatives: []



In [266]:
latex1 = r't \in [0.000, 0.0361) \rightarrow & \
U_u(\omega_t^o) \geq \max\left(Q_{U_{\Omega^o_t}} (-0.20 \cdot t + 0.22), \bar{u_t}\right)\\'
latex2 = r't \in [0.0361, 1.000] \rightarrow &  \
U_u(\omega_t^o) \geq \max\left( u, Q_{U_{\Omega^o_t}} (-0.10 \cdot t + 0.64)\right)'

In [267]:
parsed = extract_structure(latex1)

In [269]:
json_data = json.dumps(parsed, indent=2)
print(extract_semantics(json_data))

```json
{
  "Variables/Constants": [
    {"name": "t", "meaning": "A variable typically representing time."},
    {"name": "0.0361", "meaning": "A constant value, possibly a parameter or a specific measurement."},
    {"name": "1.000", "meaning": "A constant value, often used to represent a normalized or base value."},
    {"name": "u", "meaning": "A variable, which could represent a specific unknown or a parameter in an equation."},
    {"name": "0.10", "meaning": "A constant value, possibly representing a percentage or a rate in decimal form."},
    {"name": "0.64", "meaning": "A constant value, often used in calculations involving probabilities or fractions."}
  ],
  "Functions": [
    {"name": "U_u", "meaning": "A function denoted by U with a subscript u, possibly representing a utility function dependent on variable u."},
    {"name": "max", "meaning": "A mathematical function that returns the maximum value among its arguments."},
    {"name": "Q_{U_{\Omega^o_t}}", "meaning": "A f

In [270]:
print(generate_rule_based_explanation_usingGPT(extract_semantics(json_data)))

"Calculates a function \( U_u \) dependent on variable \( u \), ensuring it meets a maximum value condition \( max \) involving a specific quantitative measure \( Q \) dependent on the set \( \Omega^o_t \)."


In [271]:
reference_explanation = '''Ensures \(U_u(\omega_t^o) \) meets either a calculated statistical value or a specified minimum utility requirement in the initial interval \( [0.000, 0.0361) \).'''

In [272]:
generated_explanation = generate_rule_based_explanation_usingGPT(extract_semantics(json_data))

In [273]:
reference_tokens = [nltk.word_tokenize(reference_explanation)]
generated_tokens = nltk.word_tokenize(generated_explanation)

In [274]:
smoothie = SmoothingFunction().method4

In [275]:
bleu_score_1 = sentence_bleu(reference_tokens, generated_tokens, smoothing_function=smoothie)

In [276]:
bleu_score_1

0.06782096761471876

In [277]:
from sentence_transformers import SentenceTransformer, util

model = SentenceTransformer('bert-base-nli-mean-tokens')

# Example texts
text1 = str(reference_tokens)
text2 = str(generated_tokens)

# Convert texts to embeddings
embedding1 = model.encode([text1], convert_to_tensor=True)
embedding2 = model.encode([text2], convert_to_tensor=True)

# Compute cosine similarity
cosine_score = util.pytorch_cos_sim(embedding1, embedding2)

# Since these are single sentences, there will be only one similarity score
print(f"Cosine Similarity: {cosine_score.item():.4f}")

Cosine Similarity: 0.9437


In [278]:
text1 = 'The quick brown fox jumps over the lazy dog.'
text2 = "A fast brown fox leaps over a lazy dog."


embedding1 = model.encode([text1], convert_to_tensor=True)
embedding2 = model.encode([text2], convert_to_tensor=True)

# Compute cosine similarity
cosine_score = util.pytorch_cos_sim(embedding1, embedding2)

# Since these are single sentences, there will be only one similarity score
print(f"Cosine Similarity: {cosine_score.item():.4f}")

Cosine Similarity: 0.9610


In [279]:
text1 = 'The quick brown fox jumps over the lazy dog.'
text2 = "A fast brown fox leaps over a lazy dog."


embedding1 = model.encode([text1], convert_to_tensor=True)
embedding2 = model.encode([text2], convert_to_tensor=True)

# Compute cosine similarity
cosine_score = util.pytorch_cos_sim(embedding1, embedding2)

# Since these are single sentences, there will be only one similarity score
print(f"Cosine Similarity: {cosine_score.item():.4f}")

Cosine Similarity: 0.9610


In [280]:
from sentence_transformers import SentenceTransformer, util

# Load the SentenceTransformer model
model = SentenceTransformer('bert-base-nli-mean-tokens')

# Mathematically equivalent sentences with more advanced expressions
sentence1 = "The sum of the squares of \(a\) and \(b\) is equal to the square of \(c\), represented as \(a^2 + b^2 = c^2\)."
sentence2 = "For any right triangle, the square of the hypotenuse \(c\) equals the sum of the squares of the other two sides, expressed as \(c^2 = a^2 + b^2\)."

# Convert sentences to embeddings
embedding1 = model.encode([sentence1], convert_to_tensor=True)
embedding2 = model.encode([sentence2], convert_to_tensor=True)

# Compute cosine similarity
cosine_score = util.pytorch_cos_sim(embedding1, embedding2)

# Print the cosine similarity score
print(f"Cosine Similarity: {cosine_score.item():.4f}")


Cosine Similarity: 0.8663
